In [1]:
import os
from pathlib import Path
from mne.io import read_raw, read_raw_ant
from mne_bids import BIDSPath, write_raw_bids
from tqdm.contrib import tzip
from tqdm import tqdm
from mne import set_log_level
set_log_level("Error")

In [ ]:
root_dir = Path("/Volumes/Extreme_SSD/payam_data/Tide_data")
bids_dir = root_dir / "BIDS"


sites = ["zuerich"]

for site in sites:
    data_dir = root_dir / site
    if site in ["regensburg", "tuebingen"]:
        files = [
            f for f in data_dir.rglob("*.vhdr")
            if not any(p.startswith('.') for p in f.parts)
        ]

    if site == "illinois":
        files = [
            f for f in data_dir.rglob("*.cdt")
            if not any(p.startswith('.') for p in f.parts)
        ]
        for fname in files:
            dpo_file = fname.with_suffix('.cdt.dpo')
            dpa_file = fname.with_suffix('.cdt.dpa')
            if dpo_file.exists():
                dpo_file.rename(dpa_file)
    
    if site == "austin":
        files = [
            f for f in data_dir.rglob("*.fif")
            if not any(p.startswith('.') for p in f.parts)
        ]
    
    if site == "dublin":
        files = [
            f for f in data_dir.rglob("*_open.bdf")
            if not any(p.startswith('.') for p in f.parts)
        ]

    if site == "ghent":
        files = [
            f for f in data_dir.rglob("*.cnt")
            if not any(p.startswith('.') for p in f.parts)
        ]
        files = [f for f in files if "ses-1" in f.stem]
        
        ## convert to fif
        for fname in tqdm(files):
            raw = read_raw_ant(fname)
            raw.save(fname.with_suffix(".fif"), overwrite=True)

        files = [
            f for f in data_dir.rglob("*.fif")
            if not any(p.startswith('.') for p in f.parts)
        ]
    
    if site == "zuerich":
        bids_dir = Path("/Volumes/Extreme_SSD/payam_data/Tide_data/BIDS")
        data_dir = Path("/Volumes/G_USZ_ORL$/Research/ANTINOMICS/data/eeg")
        files = [f for f in data_dir.iterdir() if f.is_file() and f.name.endswith("_rest.vhdr")]
        files = sorted(files, key=os.path.getctime)
        subject_ids = [str(i) for i in range(70001, 70001 + len(files))]
    
    if not site == "zuerich": 
        subject_ids = [f.stem[:5] for f in files]
    
    for fname, subject_id in tzip(files, subject_ids,
                                    total=len(subject_ids),
                                    desc=f"Converting {site}"):
        raw = read_raw(fname)
        if raw.info["sfreq"] != 1000 and site == "zuerich":
            raw.resample(1000)
        
        bids_path = BIDSPath(
            subject=subject_id,
            session="01",
            task="rest",
            datatype="eeg",
            description=site,
            root=bids_dir
        )
        write_raw_bids(raw, bids_path=bids_path, overwrite=True, allow_preload=True)

Create csv ant to tide based on creation time

In [ ]:
import pandas as pd

data = []
start_id = 70001

for i, f in enumerate(files):
    old_id = f.stem[:4]
    new_id = start_id + i
    data.append((old_id, new_id))

df = pd.DataFrame(data, columns=["Antinomics_IDs", "Tide_IDs"])
df.to_csv("../material/ant_2_tide.csv", index=False)

print(df.head())
print(f"\n✅ Saved mapping for {len(df)} subjects to 'subject_id_map.csv'")

preprocess

In [21]:
from mne.io import read_raw
from mne.channels import make_standard_montage
from mne import make_fixed_length_epochs
from mne.preprocessing import ICA
from mne_icalabel import label_components

In [25]:
sfreq = 250.0
l_freq = 1.0
h_freq = 100.0
crop_duration = 5.0
epoch_duration = 10.0
overwrite = True


fname = "/Volumes/Extreme_SSD/payam_data/Tide_data/austin/10067_ses-1_rest_1.fif"
raw = mne.io.read_raw(fname, preload=True)

raw.drop_channels(ch_names="VREF")
montage = make_standard_montage("GSN-HydroCel-64_1.0")
raw.set_montage(montage, match_case=False, on_missing="warn")

## preproc_1
raw.crop(tmin=crop_duration, tmax=raw.times[-1] - crop_duration)
raw.filter(l_freq=l_freq, h_freq=h_freq)
raw.resample(sfreq=sfreq)
raw.set_eeg_reference("average", projection=False)
epochs_2 = make_fixed_length_epochs(raw, duration=10, preload=True)
# epochs_1.save(fname_1, overwrite=overwrite)

## preproc_2
# ar = AutoReject(
#                 n_interpolate=np.array([1, 4, 8]),
#                 consensus=np.linspace(0, 1.0, 11),
#                 cv=5,
#                 n_jobs=1,
#                 random_state=11,
#                 verbose=True
#                 )
# ar.fit(epochs_1)
# epochs_2 = ar.transform(epochs_1)

## preproc 3
ica = ICA(n_components=0.999, method='fastica')
try:
    ica.fit(epochs_2)
except:
    ica = ICA(n_components=5, method='infomax')
    ica.fit(epochs_2)

ic_dict = label_components(epochs_2, ica, method="iclabel")
ic_labels = ic_dict["labels"]
ic_probs = ic_dict["y_pred_proba"]
# eog_indices = [idx for idx, label in enumerate(ic_labels) \
#                 if label == "eye blink" and ic_probs[idx] > ica_label_thr]
# eog_indices_fil = [x for x in eog_indices if x <= 10]

# if len(eog_indices) > 0:
#     eog_components = ica.plot_properties(epochs_ar,
#                                             picks=eog_indices_fil,
#                                             show=False,
#                                             )
#     ica.apply(epochs_ar, exclude=eog_indices_fil)

# epochs_ar.save(fname_save_3, overwrite=overwrite)


In [26]:
ic_labels

['eye blink',
 'eye blink',
 'other',
 'brain',
 'other',
 'eye blink',
 'channel noise',
 'other',
 'eye blink',
 'eye blink',
 'eye blink',
 'channel noise',
 'other',
 'other',
 'brain',
 'other',
 'muscle artifact',
 'brain',
 'other',
 'brain',
 'heart beat',
 'other',
 'brain',
 'other',
 'brain',
 'eye blink',
 'other',
 'brain',
 'other',
 'other',
 'other',
 'brain',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'other',
 'channel noise',
 'other',
 'other',
 'other',
 'other',
 'brain',
 'other',
 'other',
 'brain',
 'other']